In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rajnishe/facescrub-full")

print("Path to dataset files:", path)

In [3]:
import os
import cv2
import numpy as np

# Initialize Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Functions
def pixelate_face(image, x, y, w, h, blocks):
    face = image[y:y+h, x:x+w]
    (fh, fw) = face.shape[:2]
    x_steps = np.linspace(0, fw, blocks + 1, dtype="int")
    y_steps = np.linspace(0, fh, blocks + 1, dtype="int")
    for i in range(1, len(y_steps)):
        for j in range(1, len(x_steps)):
            sx, sy = x_steps[j - 1], y_steps[i - 1]
            ex, ey = x_steps[j], y_steps[i]
            roi = face[sy:ey, sx:ex]
            color = roi.mean(axis=(0, 1)).astype("uint8")
            cv2.rectangle(face, (sx, sy), (ex, ey), color.tolist(), -1)
    image[y:y+h, x:x+w] = face
    return image

def blur_face(image, x, y, w, h, kernel_size):
    if kernel_size % 2 == 0:
        kernel_size += 1
    face = image[y:y+h, x:x+w]
    face_blur = cv2.GaussianBlur(face, (kernel_size, kernel_size), 0)
    image[y:y+h, x:x+w] = face_blur
    return image

# Paths
root_dir = "/kaggle/input/facescrub-full"
output_blur = "/kaggle/working/output_blur"
output_pixel = "/kaggle/working/output_pixel"
os.makedirs(output_blur, exist_ok=True)
os.makedirs(output_pixel, exist_ok=True)

# Parameters
blur_kernels = [25, 45]
pixel_blocks = [25, 45]

# Process
for subdir in ['actor_faces', 'actress_faces']:
    input_path = os.path.join(root_dir, subdir)
    for person in os.listdir(input_path):
        person_path = os.path.join(input_path, person)
        if not os.path.isdir(person_path):
            continue
        for img_file in os.listdir(person_path):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            image_path = os.path.join(person_path, img_file)
            image = cv2.imread(image_path)
            if image is None:
                continue

            # Create a small image for face detection (improve speed)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            scale = 0.5
            small_gray = cv2.resize(gray, (0, 0), fx=scale, fy=scale)
            small_faces = face_cascade.detectMultiScale(small_gray, scaleFactor=1.1, minNeighbors=5)

            # Restore to original size
            faces = [(int(x/scale), int(y/scale), int(w/scale), int(h/scale)) for (x, y, w, h) in small_faces]
            if len(faces) == 0:
                continue

            # Reuse pre-copied image
            blur_imgs = {ksize: image.copy() for ksize in blur_kernels}
            pixel_imgs = {block: image.copy() for block in pixel_blocks}

            for (x, y, w, h) in faces:
                for ksize in blur_kernels:
                    blur_imgs[ksize] = blur_face(blur_imgs[ksize], x, y, w, h, ksize)
                for block in pixel_blocks:
                    pixel_imgs[block] = pixelate_face(pixel_imgs[block], x, y, w, h, block)

            for ksize in blur_kernels:
                blur_dir = os.path.join(output_blur, f"ksize_{ksize}", subdir, person)
                os.makedirs(blur_dir, exist_ok=True)
                cv2.imwrite(os.path.join(blur_dir, img_file), blur_imgs[ksize])

            for block in pixel_blocks:
                pixel_dir = os.path.join(output_pixel, f"block_{block}", subdir, person)
                os.makedirs(pixel_dir, exist_ok=True)
                cv2.imwrite(os.path.join(pixel_dir, img_file), pixel_imgs[block])

print("\n Success!!")


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc


 Success!!


In [4]:

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Ensure output directory exists for model saving
output_dir = "/kaggle/working/models"
os.makedirs(output_dir, exist_ok=True)

# Image loading function (with special case handling)
def load_images_from_dirs(base_dirs, image_size=(64, 64), is_nested=False):
    images = []
    labels = []
    label_map = {}
    label_count = 0

    for base_dir in base_dirs:
        if is_nested:
            # Handle nested structure: /base_dir/actor_faces/Aaron_Eckhart/...
            for group_dir in os.listdir(base_dir):  # actor_faces, actress_faces
                group_path = os.path.join(base_dir, group_dir)
                if not os.path.isdir(group_path):
                    continue

                for label_name in os.listdir(group_path):  # e.g., Aaron_Eckhart
                    person_path = os.path.join(group_path, label_name)
                    if not os.path.isdir(person_path):
                        continue

                    if label_name not in label_map:
                        label_map[label_name] = label_count
                        label_count += 1
                    label = label_map[label_name]

                    for img_name in os.listdir(person_path):
                        img_path = os.path.join(person_path, img_name)
                        try:
                            img = cv2.imread(img_path)
                            if img is None:
                                continue
                            img = cv2.resize(img, image_size)
                            images.append(img)
                            labels.append(label)
                        except Exception as e:
                            print(f"Error loading image {img_path}: {e}")
                            continue
        else:
            # Original structure: /base_dir/Aaron_Eckhart/...
            for label_name in os.listdir(base_dir):
                person_path = os.path.join(base_dir, label_name)
                if not os.path.isdir(person_path):
                    continue

                if label_name not in label_map:
                    label_map[label_name] = label_count
                    label_count += 1
                label = label_map[label_name]

                for img_name in os.listdir(person_path):
                    img_path = os.path.join(person_path, img_name)
                    try:
                        img = cv2.imread(img_path)
                        if img is None:
                            continue
                        img = cv2.resize(img, image_size)
                        images.append(img)
                        labels.append(label)
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")
                        continue

    return np.array(images), np.array(labels), label_map

# CNN model definition
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Experiment settings
experiments = {
    "Original": [
        "/kaggle/input/facescrub-full/actor_faces",
        "/kaggle/input/facescrub-full/actress_faces"
    ],
    "Blur_K25": ["/kaggle/working/output_blur/ksize_25"],
    "Blur_K45": ["/kaggle/working/output_blur/ksize_45"],
    "Pixel_B25": ["/kaggle/working/output_pixel/block_25"],
    "Pixel_B45": ["/kaggle/working/output_pixel/block_45"]
}

results = {}

# Perform each experiment
for name, dirs in experiments.items():
    print(f"\n--- Running Experiment: {name} ---")

    # Check if nested structure is needed
    is_nested = name != "Original"
    X, y, label_map = load_images_from_dirs(dirs, is_nested=is_nested)

    if len(X) == 0:
        print(f"No images loaded for {name}, skipping...")
        continue

    X = X / 255.0  # Normalize
    y_cat = to_categorical(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y)

    model = create_cnn_model(X.shape[1:], y_cat.shape[1])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.1)

    # Save model for each experiment
    model_save_path = os.path.join(output_dir, f"{name.lower()}_face_model.h5")
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {name}: {test_acc:.4f}")
    results[name] = test_acc

# Result summary
print("\n=== Summary of Classification Accuracies ===")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")


--- Running Experiment: Original ---


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Epoch 1/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.0031 - loss: 6.2283 - val_accuracy: 0.0295 - val_loss: 5.4267
Epoch 2/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0540 - loss: 5.0679 - val_accuracy: 0.1283 - val_loss: 4.4013
Epoch 3/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1754 - loss: 3.9746 - val_accuracy: 0.2167 - val_loss: 3.8115
Epoch 4/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.2879 - loss: 3.2692 - val_accuracy: 0.2874 - val_loss: 3.4427
Epoch 5/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3669 - loss: 2.8123 - val_accuracy: 0.3146 - val_loss: 3.2620
Epoch 6/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4313 - loss: 2.4601 - val_accuracy: 0.3581 - val_loss: 3.0388
Epoch 7/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4837 - loss: 2.2138 - val_accuracy: 0.3740 - val_loss: 2.9542
Epoch 8/10
971/971 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5268 - loss: 1.9951 - val_accuracy: 0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.0046 - loss: 6.2013 - val_accuracy: 0.0214 - val_loss: 5.5814
Epoch 2/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0362 - loss: 5.2971 - val_accuracy: 0.0843 - val_loss: 4.8027
Epoch 3/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1011 - loss: 4.5285 - val_accuracy: 0.1278 - val_loss: 4.3132
Epoch 4/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1797 - loss: 3.9312 - val_accuracy: 0.2031 - val_loss: 3.8838
Epoch 5/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.2562 - loss: 3.4510 - val_accuracy: 0.2478 - val_loss: 3.6132
Epoch 6/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3196 - loss: 3.0937 - val_accuracy: 0.2811 - val_loss: 3.4437
Epoch 7/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3632 - loss: 2.8370 - val_accuracy: 0.3146 - val_loss: 3.3072
Epoch 8/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4080 - loss: 2.6069 - val_accuracy: 0

...

Epoch 1/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.0033 - loss: 6.2264 - val_accuracy: 0.0150 - val_loss: 5.7650
Epoch 2/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0255 - loss: 5.4876 - val_accuracy: 0.0473 - val_loss: 5.1073
Epoch 3/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0767 - loss: 4.7319 - val_accuracy: 0.1118 - val_loss: 4.5173
Epoch 4/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1479 - loss: 4.1365 - val_accuracy: 0.1715 - val_loss: 4.0932
Epoch 5/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2158 - loss: 3.6903 - val_accuracy: 0.1987 - val_loss: 3.9367
Epoch 6/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2777 - loss: 3.3624 - val_accuracy: 0.2328 - val_loss: 3.7297
Epoch 7/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3221 - loss: 3.1009 - val_accuracy: 0.2702 - val_loss: 3.5907
Epoch 8/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3534 - loss: 2.9062 - val_accuracy: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.0047 - loss: 6.2008 - val_accuracy: 0.0233 - val_loss: 5.4618
Epoch 2/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0502 - loss: 5.0838 - val_accuracy: 0.1022 - val_loss: 4.6349
Epoch 3/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1573 - loss: 4.1249 - val_accuracy: 0.2082 - val_loss: 3.8909
Epoch 4/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2907 - loss: 3.2813 - val_accuracy: 0.2779 - val_loss: 3.5716
Epoch 5/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3989 - loss: 2.7105 - val_accuracy: 0.3433 - val_loss: 3.2252
Epoch 6/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4881 - loss: 2.2727 - val_accuracy: 0.3545 - val_loss: 3.1477
Epoch 7/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5469 - loss: 1.9345 - val_accuracy: 0.4072 - val_loss: 2.9807
Epoch 8/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5969 - loss: 1.6722 - val_accuracy: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.0036 - loss: 6.2328 - val_accuracy: 0.0233 - val_loss: 5.5833
Epoch 2/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.0417 - loss: 5.2286 - val_accuracy: 0.0821 - val_loss: 4.6699
Epoch 3/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1330 - loss: 4.2436 - val_accuracy: 0.1814 - val_loss: 3.9925
Epoch 4/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.2317 - loss: 3.5798 - val_accuracy: 0.2402 - val_loss: 3.6750
Epoch 5/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3031 - loss: 3.1636 - val_accuracy: 0.2734 - val_loss: 3.3919
Epoch 6/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3574 - loss: 2.8548 - val_accuracy: 0.2989 - val_loss: 3.3158
Epoch 7/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.3966 - loss: 2.6044 - val_accuracy: 0.3405 - val_loss: 3.1404
Epoch 8/10
881/881 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4447 - loss: 2.3648 - val_accuracy: 0.

In [5]:
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import gc

# Ensure output directory exists for visualizations
output_dir = "/kaggle/working"
os.makedirs(output_dir, exist_ok=True)

# Image loader with multithreading and sampling
def load_image(img_path, image_size):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to load image: {img_path}")
        return None, None
    img = cv2.resize(img, image_size)
    label = label_map[os.path.basename(os.path.dirname(img_path))]
    return img, label

def load_original_images(base_dirs, image_size=(64, 64), max_samples=1000):
    images, labels = [], []
    paths = []
    global label_map, label_count

    for base_dir in base_dirs:
        for label_name in os.listdir(base_dir):
            person_path = os.path.join(base_dir, label_name)
            if not os.path.isdir(person_path):
                continue

            if label_name not in label_map:
                label_map[label_name] = label_count
                label_count += 1
            label = label_map[label_name]

            img_names = os.listdir(person_path)
            np.random.shuffle(img_names)
            for img_name in img_names[:max_samples // len(base_dirs)]:
                paths.append(os.path.join(person_path, img_name))
    
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda p: load_image(p, image_size), paths))
    
    for img, label in results:
        if img is not None:
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Traditional obfuscation: Pixelization
def apply_pixelization(images, block_size):
    return np.array([cv2.blur(img, (block_size, block_size)) for img in images])

# Traditional obfuscation: Gaussian blur
def apply_gaussian_blur(images, kernel_size):
    return np.array([cv2.GaussianBlur(img, (kernel_size, kernel_size), 0) for img in images])

# Differential Privacy Gaussian noise
def apply_dp_noise(images, epsilon, delta=1e-5, sensitivity=255.0):
    sigma = np.sqrt(2 * np.log(1.25 / delta)) * sensitivity / epsilon
    noisy_images = images + np.random.normal(loc=0.0, scale=sigma, size=images.shape)
    return np.clip(noisy_images, 0, 255).astype(np.uint8)

# Utility metrics (batch processing)
def compute_mse_ssim(originals, obfuscated, batch_size=100):
    mse_list, ssim_list = [], []
    for i in range(0, len(originals), batch_size):
        batch_orig = originals[i:i + batch_size]
        batch_obf = obfuscated[i:i + batch_size]
        for orig, obf in zip(batch_orig, batch_obf):
            mse_val = mean_squared_error(orig.flatten(), obf.flatten())
            ssim_val = ssim(orig, obf, channel_axis=2, data_range=255)
            mse_list.append(mse_val)
            ssim_list.append(ssim_val)
    return np.mean(mse_list), np.mean(ssim_list)

# Visualize original vs obfuscated images
def visualize_images(original, obfuscated, method, param, num_samples=5):
    indices = np.random.choice(len(original), num_samples, replace=False)
    orig_samples = original[indices]
    obf_samples = obfuscated[indices]
    plt.figure(figsize=(10, 4))
    for i in range(num_samples):
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(orig_samples[i])
        plt.title("Original")
        plt.axis("off")
        plt.subplot(2, num_samples, i + 1 + num_samples)
        plt.imshow(obf_samples[i])
        plt.title(f"{method} ({param})")
        plt.axis("off")
    plt.savefig(os.path.join(output_dir, f"comparison_{method.lower().replace(' ', '_')}_{param}.png"))
    plt.close()

# Load original images with sampling
base_dirs = ["/kaggle/input/facescrub-full/actor_faces", "/kaggle/input/facescrub-full/actress_faces"]
label_map = {}
label_count = 0
orig_images, orig_labels = load_original_images(base_dirs, max_samples=1000)
orig_images_norm = orig_images / 255.0

# Encode labels
y_cat = to_categorical(orig_labels)

# Debug: Check shapes and number of classes
print(f"y_cat shape: {y_cat.shape}")
print(f"Number of unique labels: {len(np.unique(orig_labels))}")

# Model paths for evaluation
model_paths = {
    "Original": "/kaggle/working/models/original_face_model.h5",
    "Blur_K25": "/kaggle/working/models/blur_k25_face_model.h5",
    "Blur_K45": "/kaggle/working/models/blur_k45_face_model.h5",
    "Pixel_B25": "/kaggle/working/models/pixel_b25_face_model.h5",
    "Pixel_B45": "/kaggle/working/models/pixel_b45_face_model.h5"
}

# Evaluation parameters
epsilons = [0.1, 0.5, 1.0]
pixel_block_sizes = [25, 45]  # Match Step 2
gaussian_kernels = [25, 45]   # Match Step 2

# Evaluate all models
results = {}

for model_name, model_path in model_paths.items():
    print(f"\n=== Evaluating with {model_name} Model ===")
    try:
        model = load_model(model_path)
    except Exception as e:
        print(f"Failed to load model {model_path}: {e}")
        continue

    if model.output_shape[-1] != y_cat.shape[-1]:
        print(f"Warning: Model output classes ({model.output_shape[-1]}) do not match label classes ({y_cat.shape[-1]}) for {model_name}")
        continue

    # Evaluate original dataset
    print("\n--- Original Dataset ---")
    test_loss, test_acc = model.evaluate(orig_images_norm, y_cat, verbose=0, batch_size=32)
    print(f"Classification Accuracy: {test_acc:.4f}")
    results[model_name] = {"Original": {"accuracy": test_acc * 100}}  # Store under model name

    # Evaluate NP-Pix
    print("\n--- NP-Pix (Pixelization) ---")
    for block_size in pixel_block_sizes:
        pixelized_images = apply_pixelization(orig_images, block_size)
        pixelized_images_norm = pixelized_images / 255.0
        visualize_images(orig_images, pixelized_images, "NP-Pix", f"b={block_size}")
        test_loss, test_acc = model.evaluate(pixelized_images_norm, y_cat, verbose=0, batch_size=32)
        mse_val, ssim_val = compute_mse_ssim(orig_images, pixelized_images, batch_size=32)
        print(f"\n--- Block Size = {block_size} ---")
        print(f"Classification Accuracy: {test_acc:.4f}")
        print(f"MSE: {mse_val:.2f}")
        print(f"SSIM: {ssim_val:.4f}")
        results[model_name][f"NP_Pix_b{block_size}"] = {"accuracy": test_acc * 100, "mse": mse_val, "ssim": ssim_val}
        del pixelized_images, pixelized_images_norm
        gc.collect()

    # Evaluate DP-Pix
    print("\n--- DP-Pix (Differential Privacy + Pixelization) ---")
    for epsilon in epsilons:
        for block_size in pixel_block_sizes:
            dp_images = apply_dp_noise(orig_images, epsilon)
            dp_pixelized_images = apply_pixelization(dp_images, block_size)
            dp_pixelized_images_norm = dp_pixelized_images / 255.0
            visualize_images(orig_images, dp_pixelized_images, "DP-Pix", f"b={block_size}_ε={epsilon}")
            test_loss, test_acc = model.evaluate(dp_pixelized_images_norm, y_cat, verbose=0, batch_size=32)
            mse_val, ssim_val = compute_mse_ssim(orig_images, dp_pixelized_images, batch_size=32)
            print(f"\n--- Block Size = {block_size}, ε = {epsilon} ---")
            print(f"Classification Accuracy: {test_acc:.4f}")
            print(f"MSE: {mse_val:.2f}")
            print(f"SSIM: {ssim_val:.4f}")
            results[model_name][f"DP_Pix_b{block_size}_ε{epsilon}"] = {"accuracy": test_acc * 100, "mse": mse_val, "ssim": ssim_val}
            del dp_images, dp_pixelized_images, dp_pixelized_images_norm
            gc.collect()

    # Evaluate NP-Blur
    print("\n--- NP-Blur (Gaussian Blur) ---")
    for kernel_size in gaussian_kernels:
        blurred_images = apply_gaussian_blur(orig_images, kernel_size)
        blurred_images_norm = blurred_images / 255.0
        visualize_images(orig_images, blurred_images, "NP-Blur", f"k={kernel_size}")
        test_loss, test_acc = model.evaluate(blurred_images_norm, y_cat, verbose=0, batch_size=32)
        mse_val, ssim_val = compute_mse_ssim(orig_images, blurred_images, batch_size=32)
        print(f"\n--- Kernel Size = {kernel_size} ---")
        print(f"Classification Accuracy: {test_acc:.4f}")
        print(f"MSE: {mse_val:.2f}")
        print(f"SSIM: {ssim_val:.4f}")
        results[model_name][f"NP_Blur_k{kernel_size}"] = {"accuracy": test_acc * 100, "mse": mse_val, "ssim": ssim_val}
        del blurred_images, blurred_images_norm
        gc.collect()

    # Evaluate DP-Blur
    print("\n--- DP-Blur (Differential Privacy + Gaussian Blur) ---")
    for epsilon in epsilons:
        for kernel_size in gaussian_kernels:
            dp_images = apply_dp_noise(orig_images, epsilon)
            dp_blurred_images = apply_gaussian_blur(dp_images, kernel_size)
            dp_blurred_images_norm = dp_blurred_images / 255.0
            visualize_images(orig_images, dp_blurred_images, "DP-Blur", f"k={kernel_size}_ε={epsilon}")
            test_loss, test_acc = model.evaluate(dp_blurred_images_norm, y_cat, verbose=0, batch_size=32)
            mse_val, ssim_val = compute_mse_ssim(orig_images, dp_blurred_images, batch_size=32)
            print(f"\n--- Kernel Size = {kernel_size}, ε = {epsilon} ---")
            print(f"Classification Accuracy: {test_acc:.4f}")
            print(f"MSE: {mse_val:.2f}")
            print(f"SSIM: {ssim_val:.4f}")
            results[model_name][f"DP_Blur_k{kernel_size}_ε{epsilon}"] = {"accuracy": test_acc * 100, "mse": mse_val, "ssim": ssim_val}
            del dp_images, dp_blurred_images, dp_blurred_images_norm
            gc.collect()

# Result table
print("\n=== Table 1: Accuracy (in %) of CNN Re-identification Attacks ===")
header = ["Dataset", "Original"]
header.extend([f"NP-Pix (b={b})" for b in pixel_block_sizes])
for b in pixel_block_sizes:
    header.extend([f"DP-Pix (b={b}, ε={ε})" for ε in epsilons])
header.extend([f"NP-Blur (k={k})" for k in gaussian_kernels])
for k in gaussian_kernels:
    header.extend([f"DP-Blur (k={k}, ε={ε})" for ε in epsilons])
print("| " + " | ".join(header) + " |")
print("| " + " | ".join(["-" * len(h) for h in header]) + " |")

for model_name in model_paths.keys():
    row = [model_name]
    row.append(f"{results[model_name]['Original']['accuracy']:.2f}")  # Correct key
    for b in pixel_block_sizes:
        row.append(f"{results[model_name][f'NP_Pix_b{b}']['accuracy']:.2f}")
    for b in pixel_block_sizes:
        for ε in epsilons:
            row.append(f"{results[model_name][f'DP_Pix_b{b}_ε{ε}']['accuracy']:.2f}")
    for k in gaussian_kernels:
        row.append(f"{results[model_name][f'NP_Blur_k{k}']['accuracy']:.2f}")
    for k in gaussian_kernels:
        for ε in epsilons:
            row.append(f"{results[model_name][f'DP_Blur_k{k}_ε{ε}']['accuracy']:.2f}")
    print("| " + " | ".join(row) + " |")

# MSE and SSIM summary
print("\n=== MSE and SSIM Results ===")
for model_name in model_paths.keys():
    for key, metrics in results[model_name].items():
        if "mse" in metrics:
            print(f"{model_name}_{key}:")
            print(f"  MSE: {metrics['mse']:.2f}")
            print(f"  SSIM: {metrics['ssim']:.4f}")
print("\n success!")

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Failed to load image: /kaggle/input/facescrub-full/actor_faces/Jean_Reno/Jean_Reno_54311_29031.gif


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
li

y_cat shape: (43147, 530)
Number of unique labels: 530

=== Evaluating with Original Model ===

--- Original Dataset ---
Classification Accuracy: 0.5736

--- NP-Pix (Pixelization) ---

--- Block Size = 25 ---
Classification Accuracy: 0.0089
MSE: 98.86
SSIM: 0.2689

--- Block Size = 45 ---
Classification Accuracy: 0.0037
MSE: 104.09
SSIM: 0.2238

--- DP-Pix (Differential Privacy + Pixelization) ---

--- Block Size = 25, ε = 0.1 ---
Classification Accuracy: 0.0017
MSE: 105.59
SSIM: 0.1739

--- Block Size = 45, ε = 0.1 ---
Classification Accuracy: 0.0018
MSE: 105.49
SSIM: 0.1785

--- Block Size = 25, ε = 0.5 ---
Classification Accuracy: 0.0019
MSE: 106.43
SSIM: 0.1784

--- Block Size = 45, ε = 0.5 ---
Classification Accuracy: 0.0019
MSE: 106.30
SSIM: 0.1805

--- Block Size = 25, ε = 1.0 ---
Classification Accuracy: 0.0025
MSE: 107.32
SSIM: 0.1838

--- Block Size = 45, ε = 1.0 ---
Classification Accuracy: 0.0020
MSE: 107.20
SSIM: 0.1829

--- NP-Blur (Gaussian Blur) ---

--- Kernel Size = 2